In [3]:
import sys

sys.path.insert(0, "..")

# Agents
from src.agents.agent import Agent
from src.agents.exp3 import Exp3

# Adversaries
from src.adversaries.adversary import Adversary
from src.adversaries.deceptive_adversary import DeceptiveAdversary

# Contexts
from src.contexts.context import Context
from src.contexts.gaussian_context import GaussianContext

# Environments
from src.environments.environment import Environment
from src.environments.adversarial_mab_env import Adversarial_MAB_env, Random_Adversarial_MAB_env
from src.environments.adversarial_contextual_env import AdversarialContextualEnv

# Adversarial Contextual MAB bandits
from src.adversarial_contextual_mab import AdversarialContextualMAB

# Display
from src.display.display import plot_result

In [ ]:
# Init vars
K = 10
h = 1000
num_sim = 100
num_ctx = 5
num_ftrs = 3

# Init Adversary
adv = DeceptiveAdversary(K=K)

# Init Context
ctx = GaussianContext(n_contexts=num_ctx, n_arms=K, n_features=num_ftrs)

# Init Agent
ag = 

# Init Environment
adv_env = AdversarialContextualEnv(adversary=adv, context=ctx)

# Init Experiment
exp = AdversarialContextualMAB(agent=ag, environment=adv_env, num_sim=num_sim, horizon=h)